# Using pulp solver to test the given problem.
#Failed because the problem needs a cplex license to run or a solver installed.
#new : by highpy, we can use this solver to solve the problem. However, it is different from the cplex solver.  

# Solving an Optimization Problem with HiGHS via PuLP

This example demonstrates how to use Python to define a linear programming problem using **PuLP** and solve it using the **HiGHS** solver through the `highspy` interface. The code defines a simple maximization problem with decision variables, an objective function, and several constraints, then exports the model to an MPS file and solves it using HiGHS.

## Mathematical Formulation

The optimization problem is formulated as follows:

### Objective Function
Maximize:
$$
Z = 1x_1 + 2x_2 + 3x_3 + 1x_4
$$

### Subject to Constraints
$$
-c_1: -x_1 + x_2 + x_3 + 10x_4 \leq 20
$$
$$
-c_2: x_1 - x_2 + x_3 \leq 30
$$
$$
-c_3: x_2 - 3.5x_4 = 0
$$

### Variable Bounds
$$
0 \leq x_1 \leq 40, \quad 0 \leq x_2, \quad 0 \leq x_3, \quad 2 \leq x_4 \leq 3
$$
$$
x_1, x_2, x_3 \in \mathbb{R}, \quad x_4 \in \mathbb{Z}
$$

In [1]:
import numpy as np
import pulp
import highspy


# Define the problem
my_obj = [1, 2, 3, 1]
my_ub = [40, None, None, 3]
my_lb = [0, 0, 0, 2]
my_ctype = ["Continuous", "Continuous", "Continuous", "Integer"]
my_colnames = ["x1", "x2", "x3", "x4"]
my_rhs = [20, 30, 0]
my_rownames = ["c1", "c2", "c3"]
my_sense = ["<=", "<=", "="]

# Create the problem
prob = pulp.LpProblem("Maximize_Objective", pulp.LpMaximize)

# Create decision variables
variables = []
for i, name in enumerate(my_colnames):
    var_type = pulp.LpContinuous if my_ctype[i] == "Continuous" else pulp.LpInteger
    var = pulp.LpVariable(name, lowBound=my_lb[i], upBound=my_ub[i], cat=var_type)
    variables.append(var)

# Set the objective function
prob += pulp.lpDot(my_obj, variables), "Objective"

# Add constraints
constraints = [
    ([-1, 1, 1, 10], "<=", 20),
    ([1, -1, 1, 0], "<=", 30),
    ([0, 1, 0, -3.5], "=", 0)
]

for i, (coeffs, sense, rhs) in enumerate(constraints):
    expr = pulp.lpDot(coeffs, variables)
    if sense == "<=":
        prob += (expr <= rhs), my_rownames[i]
    elif sense == ">=":
        prob += (expr >= rhs), my_rownames[i]
    elif sense == "=":
        prob += (expr == rhs), my_rownames[i]

# Export the problem to an MPS file for HiGHS
mps_filename = "model.mps"
prob.writeMPS(mps_filename)

# Solve the problem using HiGHS via highspy
h = highspy.Highs()
h.readModel(mps_filename)
h.run()

# Print the solution status
print('Model', mps_filename, 'has status', h.getModelStatus())

# Retrieve and print the variable values
solution = h.getSolution().col_value
for i, var in enumerate(variables):
    print(f"{var.name} =", solution[i])  # 获取变量的最终取值


Model model.mps has status HighsModelStatus.kOptimal
x1 = 7.0
x2 = 7.0
x3 = 0.0
x4 = 2.0


In [2]:
import numpy as np
from pyscipopt import Model

# 初始化模型
model = Model("Maximize_Objective")

# 定义变量
variables = []
for i, name in enumerate(my_colnames):
    # 根据变量类型选择连续型或整数型
    vtype = "C" if my_ctype[i] == "Continuous" else "I"
    var = model.addVar(name=name, vtype=vtype, lb=my_lb[i], ub=my_ub[i])
    variables.append(var)

# 设置目标函数
model.setObjective(
    sum(my_obj[i] * variables[i] for i in range(len(variables))),
    "maximize"
)

# 添加约束
constraints = [
    ([-1, 1, 1, 10], "<=", 20),
    ([1, -1, 1, 0], "<=", 30),
    ([0, 1, 0, -3.5], "=", 0)
]

for i, (coeffs, sense, rhs) in enumerate(constraints):
    expr = sum(coeffs[j] * variables[j] for j in range(len(coeffs)))
    if sense == "<=":
        model.addCons(expr <= rhs, name=my_rownames[i])
    elif sense == ">=":
        model.addCons(expr >= rhs, name=my_rownames[i])
    elif sense == "=":
        model.addCons(expr == rhs, name=my_rownames[i])

# 求解模型
model.optimize()

# 检查求解状态并打印结果
if model.getStatus() == "optimal":
    print("Optimal solution found:")
    for var in variables:
        print(f"{var.name} =", model.getVal(var))
else:
    print("No optimal solution found. Status:", model.getStatus())


Optimal solution found:
x1 = 30.5
x2 = 10.5
x3 = 10.0
x4 = 3.0
